In [10]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

# TCRD Intro (Appyter)

* http://juniper.health.unm.edu/tcrd/
* http://juniper.health.unm.edu/tcrd/api.html
* https://pharos.nih.gov/idg/api

In [11]:
import sys, json, urllib.request
import pandas as pd
import matplotlib.pyplot
import mysql.connector as mysql

In [12]:
DBHOST = "tcrd.kmc.io"
DBNAME = "tcrd6110"
DBUSR = "tcrd"

In [13]:
dbcon = mysql.connect(host=DBHOST, db=DBNAME, user=DBUSR)

### Db Info

In [14]:
pd.read_sql("SELECT * FROM dbinfo", dbcon).transpose()

,0
dbname,tcrd
schema_ver,6.6.2
data_ver,6.11.0
owner,smathias
is_copy,0
dump_file,None


### Select target

In [8]:
%%appyter code_exec

{% do SectionField(name='target_selection',
        title='Target Selection',
        subtitle='Choose between gene symbol or UniprotId.')%}


{% set target_field = TabField(
    name='target_field',
    label='Target Selection',
    default='Gene symbol',
    description='Select input method',
    choices={
        'Gene symbol': [
            StringField(
                name='input_symbol',
                label='Gene symbol',
                default='CDK1',
                description='Input your gene symbol of interest.',
                section = 'target_selection'
            ),

        ],
        
        'UniprotId': [
                StringField(
                name='input_uniprot',
                label='UniprotId',
                default='P06493',
                description='Input your UniprotId of interest.',
                section = 'target_selection'
            ),
        ],
    },
    section = 'target_selection'
) %}

In [1]:
%%appyter code_exec

{% set uniprotId, geneSymbol = '', '' %}
{% if target_field.raw_value == 'UniprotId' %}
{% set uniprotId = target_field.value[0] %}
{% else %}
{% set geneSymbol = target_field.value[0] %}
{% endif %}

{% if uniprotId != '' %}

sql = """SELECT * FROM target
LEFT OUTER JOIN t2tc ON t2tc.target_id = target.id
LEFT OUTER JOIN protein p ON p.id = t2tc.protein_id
WHERE p.uniprot = {{uniprotId}}"""

{% else %}

sql = """SELECT * FROM target
LEFT OUTER JOIN t2tc ON t2tc.target_id = target.id
LEFT OUTER JOIN protein p ON p.id = t2tc.protein_id
WHERE p.sym = {{geneSymbol}}"""

{% endif %}

df = pd.read_sql(sql, dbcon)
df.transpose()

UsageError: Cell magic `%%appyter` not found.
